In [1]:
%use adventOfCode

In [2]:
val aoc = AocClient.fromEnv().interactiveDay(2025, 7)
aoc.viewPartOne()

--- Day 7: Laboratories --- 
 You thank the cephalopods for the help and exit the trash compactor, finding yourself in the familiar halls of a North Pole research wing. 
 Based on the large sign that says "teleporter hub", they seem to be researching teleportation ; you can't help but try it for yourself and step onto the large yellow teleporter pad. 
 Suddenly, you find yourself in an unfamiliar room! The room has no doors; the only way out is the teleporter. Unfortunately, the teleporter seems to be leaking magic smoke . 
 Since this is a teleporter lab, there are lots of spare parts, manuals, and diagnostic equipment lying around. After connecting one of the diagnostic tools, it helpfully displays error code 0H-N0 , which apparently means that there's an issue with one of the tachyon manifolds . 
 You quickly locate a diagram of the tachyon manifold (your puzzle input). A tachyon beam enters the manifold at the location marked S ; tachyon beams always move downward . Tachyon beams pass freely through empty space ( . ). However, if a tachyon beam encounters a splitter ( ^ ), the beam is stopped; instead, a new tachyon beam continues from the immediate left and from the immediate right of the splitter. 
 For example: 
 .......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
 
 In this example, the incoming tachyon beam ( | ) extends downward from S until it reaches the first splitter: 
 .......S.......
.......|.......
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
 
 At that point, the original beam stops, and two new beams are emitted from the splitter: 
 .......S.......
.......|.......
......|^|......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
 
 Those beams continue downward until they reach more splitters: 
 .......S.......
.......|.......
......|^|......
......|.|......
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
 
 At this point, the two splitters create a total of only three tachyon beams, since they are both dumping tachyons into the same place between them: 
 .......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
 
 This process continues until all of the tachyon beams reach a splitter or exit the manifold: 
 .......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
 
 To repair the teleporter, you first need to understand the beam-splitting properties of the tachyon manifold. In this example, a tachyon beam is split a total of 21 times. 
 Analyze your manifold diagram. How many times will the beam be split?

In [3]:
val exampleInput = """
.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............""".trim()

exampleInput

.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............

In [4]:
@JvmInline
value class Row(val row: String) {
    override fun toString() = row
}

data class Manifold(val rows: List<Row>, val beamIndex: Int = 1, val splitCount: Int = 0) {
    override fun toString(): String = rows.joinToString("\n")
}

fun String.parse(): Manifold = Manifold(this.trim().lines().map(::Row))


val Char.isBeam
    get() = this == '|' || this == 'S'

val Char.isSplitter
    get() = this == '^'

val Char.isEmpty
    get() = this == '.'


fun Row.biem(previousRow: Row, onSplit: () -> Unit): Row {
    val result = this.row.toCharArray()
    for (i in result.indices) {
        when {
            result[i].isSplitter && previousRow.row[i].isBeam -> {
                // looking at the input I don't suspect this to go out of bounds
                if (result[i - 1].isEmpty) result[i - 1] = '|'
                if (result[i + 1].isEmpty) result[i + 1] = '|'
                onSplit()
            }

            result[i].isEmpty && previousRow.row[i].isBeam -> {
                result[i] = '|'
            }
        }
    }
    return Row(result.concatToString())
}

fun Manifold.biem(): Manifold? {
    val previousRow = rows[beamIndex - 1]
    val row = rows.getOrNull(beamIndex) ?: return null
    var splitCount = this.splitCount
    val newRow = row.biem(previousRow) { splitCount++ }
    val rows = rows.toMutableList()
    rows[beamIndex] = newRow
    return Manifold(
        rows = rows, beamIndex = beamIndex + 1, splitCount = splitCount
    )

}

fun Manifold.totalSplitCount(onEach: (Manifold) -> Unit = {}): Int  {
    var mf = this
    while (true) {
        onEach(mf)
        mf = mf.biem() ?: break
    }
    return mf.splitCount
}

In [5]:
val exampleManifold = exampleInput.parse()
exampleManifold.totalSplitCount()

21

In [6]:
val input = aoc.input().parse()
input.totalSplitCount()

1660

In [7]:
//aoc.submitPartOne(input.totalSplitCount())

In [8]:
aoc.viewPartTwo()

--- Part Two --- 
 With your analysis of the manifold complete, you begin fixing the teleporter. However, as you open the side of the teleporter to replace the broken manifold, you are surprised to discover that it isn't a classical tachyon manifold - it's a quantum tachyon manifold . 
 With a quantum tachyon manifold, only a single tachyon particle is sent through the manifold. A tachyon particle takes both the left and right path of each splitter encountered. 
 Since this is impossible, the manual recommends the many-worlds interpretation of quantum tachyon splitting: each time a particle reaches a splitter, it's actually time itself which splits. In one timeline, the particle went left, and in the other timeline, the particle went right. 
 To fix the manifold, what you really need to know is the number of timelines active after a single particle completes all of its possible journeys through the manifold. 
 In the above example, there are many timelines. For instance, there's the timeline where the particle always went left: 
 .......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
...|^.^...^....
...|...........
..|^.^...^.^...
..|............
.|^...^.....^..
.|.............
|^.^.^.^.^...^.
|..............
 
 Or, there's the timeline where the particle alternated going left and right at each splitter: 
 .......S.......
.......|.......
......|^.......
......|........
......^|^......
.......|.......
.....^|^.^.....
......|........
....^.^|..^....
.......|.......
...^.^.|.^.^...
.......|.......
..^...^|....^..
.......|.......
.^.^.^|^.^...^.
......|........
 
 Or, there's the timeline where the particle ends up at the same point as the alternating timeline, but takes a totally different path to get there: 
 .......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
....^|^...^....
.....|.........
...^.^|..^.^...
......|........
..^..|^.....^..
.....|.........
.^.^.^|^.^...^.
......|........
 
 In this example, in total, the particle ends up on 40 different timelines. 
 Apply the many-worlds interpretation of quantum tachyon splitting to your manifold diagram. In total, how many different timelines would a single tachyon particle end up on?

In [13]:
fun Manifold.totalTimeLines(): Long {

    val cache = mutableMapOf<Pair<Int, Int>, Long>() // without this, my laptop melts ;p

    fun recurse(beamRow: Int, beamIndex: Int): Long {
        if (beamRow + 1 > rows.lastIndex) return 1L
        cache[beamRow to beamIndex]?.let { return it }

        val nextCharDown = rows[beamRow + 1].row[beamIndex]

        return when {
            nextCharDown.isEmpty -> recurse(beamRow + 1, beamIndex)
            nextCharDown.isSplitter -> recurse(beamRow + 1, beamIndex - 1) + recurse(beamRow + 1, beamIndex + 1)
            else -> error("")
        }.also {
            cache[beamRow to beamIndex] = it
        }
    }

    return recurse(0, rows.first().row.indexOfFirst { it.isBeam })
}

In [14]:
exampleInput.parse().totalTimeLines()

40

In [15]:
aoc.input().parse().totalTimeLines()

305999729392659

In [16]:
aoc.submitPartTwo(aoc.input().parse().totalTimeLines())

Your answer: 305999729392659. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 7! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .